In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt

In [2]:
lang = ["ita", "eng"]
for i_lang in lang:
    dataframes = []
    directory = "./dataset-eval/"
    for file in os.listdir(directory):
        if file.endswith(f"{i_lang}.tsv"):
            file_path = os.path.join(directory, file)
            df = pd.read_csv(file_path, sep='\t')
            dataframes.append(df)
    
    # Unisci tutti i DataFrame
    combined_df = pd.concat(dataframes, ignore_index=True)
    # Ordina
    sorted_df = combined_df.sort_values(by='Percorso', ascending=True)
    # Salva
    sorted_df.to_csv(f"all_outputs_{i_lang}.csv", sep="|")

In [3]:
sorted_df.columns

Index(['ID', 'Algoritmo', 'Percorso', 'Dataset', 'Città', 'Soggetto_originale',
       'Risposta 1', 'Valutazione 1 risposta', 'Città_1', 'Soggetto_1',
       'Descrizione_1', 'Risposta 2', 'Valutazione 2 risposta', 'Soggetto_2',
       'Descrizione_2', 'Note'],
      dtype='object')

In [4]:
cols = ['Città_1', 'Soggetto_1', 'Descrizione_1', 'Soggetto_2', 'Descrizione_2']

In [5]:
lang = ["ita", "eng"]
for i_lang in lang:
    print(i_lang)
    df = pd.read_csv(f"all_outputs_{i_lang}.csv", sep="|")
    for i in cols:
        print(i, df[i].unique())
    print("\n", df[df["Città_1"].isna()])
    print(df[df["Soggetto_1"].isna()])
    print(df[df["Descrizione_1"].isna()])
    print("\n\n")

ita
Città_1 ['NO' 'OK' 'ERR' 'PAR']
Soggetto_1 ['NO' 'OK' 'ERR']
Descrizione_1 ['NO' 'OK' 'ERR']
Soggetto_2 ['ERR' 'NO' 'OK' nan]
Descrizione_2 ['ERR' 'OK' 'NO' nan]

 Empty DataFrame
Columns: [Unnamed: 0, ID, Algoritmo, Percorso, Dataset, Città, Soggetto_originale, Risposta 1, Valutazione 1 risposta, Città_1, Soggetto_1, Descrizione_1, Risposta 2, Valutazione 2 risposta, Soggetto_2, Descrizione_2, Note]
Index: []
Empty DataFrame
Columns: [Unnamed: 0, ID, Algoritmo, Percorso, Dataset, Città, Soggetto_originale, Risposta 1, Valutazione 1 risposta, Città_1, Soggetto_1, Descrizione_1, Risposta 2, Valutazione 2 risposta, Soggetto_2, Descrizione_2, Note]
Index: []
Empty DataFrame
Columns: [Unnamed: 0, ID, Algoritmo, Percorso, Dataset, Città, Soggetto_originale, Risposta 1, Valutazione 1 risposta, Città_1, Soggetto_1, Descrizione_1, Risposta 2, Valutazione 2 risposta, Soggetto_2, Descrizione_2, Note]
Index: []



eng
Città_1 ['NO' 'ERR' 'OK' 'PAR']
Soggetto_1 ['NO' 'OK' 'ERR']
Descrizione_1 

# SHOW images

In [2]:
subjects = pd.read_csv("dataset.csv", sep=';')

In [3]:
from matplotlib.image import imread
from PIL import Image
import numpy as np

In [14]:
def carica_immagini_da_cartella(cartella, dataset, subjects, dimensione_thumbnail=(200, 200)):
    immagini = []
    nomi_file = []
    for file in sorted(os.listdir(cartella)):
        if file.lower().endswith(('png', 'jpg', 'jpeg')) and dataset in file and "portici" not in file:
            percorso_file = os.path.join(cartella, file)
            immagine = Image.open(percorso_file).resize(dimensione_thumbnail)
            immagini.append(np.array(immagine))

            if "i17" in file:
                file_subject = subjects[subjects.id == 17].subject.item()
                file_subject = "Torri"
            elif "i12" in file:
                file_subject = subjects[subjects.id == 12].subject.item() + " - Milano"
            else:
                file_subject = subjects[subjects.id == int(file.split("_")[0][1:])].subject.item()
            # print(file, file_subject)
            nomi_file.append(file_subject)
                
    # print(dataset, len(immagini), len(nomi_file))
    return immagini, nomi_file

# Datasets di esempio
datasets = {
    "FloreView": carica_immagini_da_cartella("./dataset/", "flore", subjects),
    "Wikimedia": carica_immagini_da_cartella("./dataset/", "wiki", subjects),
    "Other": carica_immagini_da_cartella("./dataset/", "other", subjects),
}

flore 10 10
wiki 20 20
other 10 10


In [30]:
dati_img = []
for key in datasets.keys():
    dati_img.extend(datasets[key][0])
    
dati_label = []
for key in datasets.keys():
    dati_label.extend(datasets[key][1])

In [32]:
# Define the number of rows and columns per image
rows = 8
cols = 5

# Iterate over the rows and save each row as a separate image
for row in range(rows):
    fig, axes = plt.subplots(nrows=1, ncols=cols, figsize=(15, 4))  # Adjust height for a single row
    
    # Load and display the images for the current row
    for col in range(cols):
        idx = row * cols + col
        if idx < len(dati_img):  # Ensure we don't exceed the number of images
            axes[col].imshow(dati_img[idx], aspect="equal")
            axes[col].set_title(dati_label[idx], fontsize=12, y=-0.1)
            axes[col].axis('off')  # Hide the axes
        else:
            axes[col].axis('off')  # Hide empty axes if images are fewer
            
    # Adjust layout and save the figure for the current row
    plt.tight_layout()
    plt.savefig(f'row_{row + 1}.pdf', dpi=300)  # Save with a unique filename
    plt.close(fig)  # Close the figure to free memory
